In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sbn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Mercedes file selected
df = pd.read_csv("/kaggle/input/used-car-dataset-ford-and-mercedes/merc.csv")

In [3]:
df.head()

In [4]:
df.describe()

In [5]:
# checks if it has a null cell
df.isnull().sum()

In [6]:
plt.figure(figsize=(7,5))
sbn.distplot(df["price"])
#some of cars are very expensive

In [7]:
sbn.countplot(df["year"])

In [8]:
df.corr()

In [9]:
df.corr()["price"].sort_values() #which effects most

In [10]:
sbn.scatterplot(x = "mileage", y = "price", data = df) # feature, price

In [11]:
df.sort_values("price",ascending = False).head(20) #higher to lower, expensive cars

In [12]:
len(df) # if I take 99% of date It wont be anything

In [13]:
len(df)*0.01

In [14]:
# create new data frame which has %99 of data from df
ninetyNine = df.sort_values("price",ascending = False).iloc[131:]

In [15]:
ninetyNine.describe()

In [16]:
plt.figure(figsize=(7,5))
sbn.displot(ninetyNine["price"])

In [17]:
df.groupby("year").mean()["price"]

In [18]:
ninetyNine.groupby("year").mean()["price"]

In [19]:
df[df.year != 1970].groupby("year").mean()["price"]

In [20]:
df = ninetyNine

In [21]:
# 1970 has bad prices
df = df[df.year != 1970]

In [22]:
df.head(20)

**String to Integers**

In [23]:
# It can be use when some of futures (model, transmission, fuelType) are needed.
"""from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

df['model']= label_encoder.fit_transform(df['model']) 
df['transmission']= label_encoder.fit_transform(df['transmission']) 
df['fuelType']= label_encoder.fit_transform(df['fuelType']) 

df.head()"""

In [24]:
#df["model"].cat.set_categories(["A Class", "B Class", "C Class", "CL Class", "CLA Class", "CLC Class", "CLK", "CLS Class", "E Class", "G Class", "GL Class", "GLA Class", "GLB Class", "GLC Class", "GLE Class", "GLS Class", "M Class","R Class", "S Class", "SL CLASS", "SLK", "V Class", "X-CLASS", "180", "200", "220", "230"], ordered = True, inplace = True)

In [25]:
# I don't use them
df = df.drop("model", axis=1)
df = df.drop("transmission", axis=1)
df = df.drop("fuelType", axis=1)

In [26]:
# y has prices
y = df["price"].values
x = df.drop("price", axis = 1) # take all features except price

In [27]:
df.head()

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
# to test data I used sklearn train test split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 10)

In [30]:
len(x_train)

In [31]:
len(x_test)

In [32]:
from sklearn.preprocessing import MinMaxScaler

In [33]:
#to make Data between 0 to 1. Better results
scaler = MinMaxScaler()

In [34]:
x_train = scaler.fit_transform(x_train)

In [35]:
x_test = scaler.fit_transform(x_test)

In [36]:
# Tensorflow used across a range of tasks but has a particular focus on training and inference of deep neural networks. 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [37]:
x_train.shape

In [38]:
model = Sequential()

# 4 layers
model.add(Dense(12, activation="relu")) # number of unit is 12
model.add(Dense(12, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(12, activation="relu"))

"""ReLu -> The function returns 0 if it receives any negative input, but for any positive value  x  
it returns that value back. So it can be written as  f(x)=max(0,x) 
"""
model.add(Dense(1)) # for output

model.compile(optimizer = "adam", loss = "mse")

In [39]:
model.fit(x = x_train, y = y_train, validation_data = (x_test, y_test), batch_size = 250, epochs = 300)

In [40]:
lossData = pd.DataFrame(model.history.history)

In [41]:
lossData.head()

In [42]:
lossData.plot()

In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [44]:
lossData = model.predict(x_test)

In [45]:
lossData

In [46]:
mean_absolute_error(y_test, lossData) #how much error in price

In [47]:
df.describe() #price mean is 24k error is 3223 / 24000 = 13% difference

In [48]:
plt.scatter(y_test, lossData)
plt.plot(y_test, y_test,"g-*")

In [49]:
# second car
df.iloc[2]

In [50]:
# new car futures came from second car .
newCarSeries = df.drop("price",axis=1).iloc[2] 

In [51]:
df.iloc[2]

In [52]:
newCarSeries = scaler.transform(newCarSeries.values.reshape(-1,5))

In [53]:
model.predict(newCarSeries) #new car price is 60670 instead of 65980